# 预处理训练和测试数据

### 读入数据集

In [13]:
with open('./data/RMRB_v3.txt', 'r', encoding='UTF-8') as f:
    raw = f.read()
# with open('./data_old2_modify.txt', 'w', encoding='utf-8') as w:
#     s = set()
#     for line in raw:
#         line = line.strip()
#         if line:
# #             w.write(line.strip()[-1]+'\n',)
#             if line[-1] not in ['\.','。','\!','！','\?','？','，','\,']:
#                 line = line+'。'
#                 w.write(line+'\n')
#             else:
#                 w.write(line+'\n')
                
# #     print(s)

- `.`不用管
- `。`不用管
- `！`不用管
- `？`不用管

[\\.。\\!！\\?？，\\,]

## 去除空格、章节

- 古诗网站爬取的97万字的诗词

In [14]:
# 总字数
print(len(raw))

123499871


In [15]:
raw[0:10]

'2017年12月31'

In [16]:
import time
import re

print(len(raw))
# ptn_1 = re.compile("(---|---第[\u96f6\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341\u767e\u5343\u4e07]{1,4}章|第[\u96f6\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341\u767e\u5343\u4e07]{1,4}章)")
# raw = ptn_1.sub(r"", raw)
raw = raw.replace(" ", "").replace("\u3000", "")
# print(len(raw))

123499871


## 查看文集中的所有符号

In [17]:
from zhon.hanzi import punctuation

# 输出标点符号正则匹配
print(r'[{}]'.format(punctuation))

[＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､　、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。]


In [18]:
print(len(raw))
punc1 = set(re.findall(r'[{}]'.format(punctuation), raw))
punc2 = set(re.findall(r'[\!\"\#\$\%\&\'\(\)\*\+,\-./:;\<=\>\?@\[\\\]\^\_\`\{\|\}\~\[\!\"\#\$\%\&\'\(\)\*\+,\-./:;\<=\>\?@\[\\\]\^\_\`\{\|\}\~\■□X]', raw))
punc = punc1 | punc2
print("punctuation of train: ", punc)

123499871
punctuation of train:  {'＠', '：', '_', "'", '％', '@', '＞', '＋', '#', '.', '＝', '！', '<', '/', '+', '＜', '*', '。', ':', '，', '&', '？', '=', '《', '［', '＆', '—', '＊', '-', 'X', '］', '%', '、', '·', '>', '》', '－', '／'}


---
从上面的符号匹配，分析后，需要对原来
- ~~[，。！？、] 不处理~~
- （）() 内容删除
- ~~> 大于号，保留~~
- ~~< 小于号不删除~~
- 〈〉 删除掉
- ~~· 不处理---~~
- ~~& 不处理~~
- ~~－ 不处理~~
- ~~+ 不处理~~
- [] 内容删除
- 【】内容删除
- <> 组合在一起时 删除掉<>,使用group
- ~~＆ 不处理~~
- ’‘ 变为顿号
- ‘’删除
- ?变？
- ~~？不变~~
- ~~！不变~~
- ~~—— 不变~~
- ~~＝ 不处理~~
- ~~＞ 大于号不处理~~
- 〔〕内容删除
- ~~.不处理~~
- …… 变为句号
- ~~＜ 小于号 不处理~~
- ~~：不处理~~
- “”删除
- ”“ 变顿号
- ； 变句号
- ﹑ 变 、号
- ! 变 ！号
- , 变，，
- 《》暂时不处理


- \[`~!@#$%\^&\*\(\)\_\-\=\+\\\|;:'",<.>/?{}\]
- \[\!\"\#\$\%\&\'\(\)\*\+,\-./:;\<=\>\?@\[\\\]\^\_\`\{\|\}\~\]

### 关于英文标点符号，使用string.punctuation

In [19]:
import string
r'[{}]'.format(string.punctuation)

'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

## 开始去除各种奇怪符号，保留叹号、逗号、问号、句号、未知X、冒号

In [20]:
import os

# {'.', '—', '《', '“', '？', '。', '【', '＿', '】', 'X', '■', '；', '》', '、', '…', '?', '：', '！', '”', '!', ',', '"', '，'}
def modify_data(data):
    """
    去除或修改指定数据集中的标点符号
    """
#     assert os.path.exists(path), "输入路径不存在"
#     with open(path, 'r', encoding='UTF-8') as f:
#         data = f.read()
    #     print(type(data))
    #     print(r'[{}]'.format(punctuation))
#         pctns = set(re.findall(r'[{}]'.format(punctuation), data))
    #     print("punctuation of test: ", pctns)
    ptn = re.compile('（.*?）')  # 删除（）() 内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('\(.*?\)')  # 删除（）() 内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('\(.*?\）')  # 删除(） 内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('\（.*?\)')  # 删除（) 内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('〈')  # 删除〈内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('〉')  # 删除 〉内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile('\[.*?\]')  # 删除[]内容［
    data = ptn.sub(r'', data)
    
    ptn = re.compile('［.*?］')  # 删除[]内容［
    data = ptn.sub(r'', data)
    
    ptn = re.compile('【.*?】')  # 删除【】内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile(r'(<)(.*?)(>)')   # 删除<>保留中间
    data = ptn.sub(r'\2', data)
    
    ptn = re.compile(r'’‘')   # ’‘ 变为顿号
    data = ptn.sub(r'、', data)
    
    ptn = re.compile(r'‘')   # 删除‘保留中间
    data = ptn.sub(r'', data)
    
    ptn = re.compile(r'’')   # 删除’保留中间
    data = ptn.sub(r'', data)
    
    ptn = re.compile(r'\?')   # ?变？
    data = ptn.sub(r'？', data)
    
    ptn = re.compile(r'”“')   # ”“ 变为顿号
    data = ptn.sub(r'、', data)
    
    ptn = re.compile(r'“')   # 删除“保留中间
    data = ptn.sub(r'', data)
    
    ptn = re.compile(r'”')   # 删除”保留中间
    data = ptn.sub(r'', data)
    
    ptn = re.compile('〔.*?〕')  # 删除〔〕内容
    data = ptn.sub(r'', data)
    
    ptn = re.compile(r'……')   # ……替换为句号
    data = ptn.sub(r'。', data)
    
    ptn = re.compile(r'…')   # …替换为句号
    data = ptn.sub(r'。', data)
    
    ptn = re.compile(r'；')   # ；替换为句号
    data = ptn.sub(r'。', data)
    
    ptn = re.compile(r';')   # ;替换为句号
    data = ptn.sub(r'。', data)
    
    ptn = re.compile(r'﹑')   # ﹑替换为、
    data = ptn.sub(r'、', data)
    
    ptn = re.compile(r'!')   # !替换为！
    data = ptn.sub(r'！', data)
    
    ptn = re.compile(r',')   # ,替换为，
    data = ptn.sub(r'，', data)
    
    ptn = re.compile(r'�')   # �替换为空
    data = ptn.sub(r'', data)
    
#     ptn = re.compile(r'^——')   # 开头的^——删除
#     data = ptn.sub(r'', data)

    return data

In [21]:
# 修改数据
raw = modify_data(raw)

In [22]:
len(raw)

123498131

In [23]:
with open('./data/RMRB_v3.txt', 'w', encoding='utf-8') as w:
    print(raw, file=w)

## 检测文本的修改结果

In [24]:
punc1 = set(re.findall(r'[{}]'.format(punctuation), raw))
punc2 = set(re.findall(r'[\!\"\#\$\%\&\'\(\)\*\+,\-./:;\<=\>\?@\[\\\]\^\_\`\{\|\}\~\[\!\"\#\$\%\&\'\(\)\*\+,\-./:;\<=\>\?@\[\\\]\^\_\`\{\|\}\~\■□X]', raw))
punc = punc1 | punc2
print(punc)

{'＠', '：', '_', "'", '％', '@', '＞', '＋', '#', '.', '＝', '！', '<', '/', '+', '＜', '*', '。', ':', '，', '&', '？', '=', '《', '＆', '—', '＊', '-', 'X', '%', '、', '·', '>', '》', '－', '／'}


只有五种必须标点，。？！、

## 获取书名号中的词，作为分词单位

为了避免未处理的符号问题，将获取书名放在了去除标点之后

In [25]:
# 测试正则分组匹配部分内容
ptn_tmp = re.compile('(《)(.+?)(》)')
out = re.finditer(ptn_tmp, raw)
vocab = []
for i in out:
    vocab.append(i.group(2))
print(len(vocab))
with open('vocab_extra_log', 'w', encoding='utf-8') as w:
    print(vocab, file=w)

130602


In [26]:
# 去除重复
vocab = set(vocab)
print(len(vocab))
with open('vocab_extra_log', 'w', encoding='utf-8') as w:
    print(vocab, file=w)

23696


In [6]:
# 新的匹配方法:命名组(?P<>要匹配的部分)，测试
ptn_tmp = re.compile('《(?P<abc>.*?)》')
out = re.finditer(ptn_tmp, raw)
vocab = []
for i in out:
    vocab.append(i.group('abc'))
print(vocab)

['柏舟', '采芑', '梁父吟', '论语', '周礼', '尧典', '尚书大传', '秣陵', '吴趋', '水调', '白雪', '送穷文', '玲珑', '楞伽', '柘枝', '子虚', '吴趋', '登楼赋', '吴趋', '水调', '白雪', '维摩品', '浣溪沙', '行路难', '葛洪传', '兰亭', '黄庭', '杨柳枝', '柘枝', '竹枝', '长门赋', '典', '坟', '想夫怜', '阳关', '白翎', '步虚词', '小游仙', '黄庭', '哨遍', '潜夫', '白苎', '典', '坟', '玉树', '望江南', '郁轮流袍', '金缕', '黄庭', '易经', '西厢', '集中录', '水仙子', '西湖四时渔歌']


In [27]:
# 查看其中的符号
from zhon.hanzi import punctuation as zh_punc

temp = ''.join(vocab)
ptn = re.compile('['+zh_punc+']')
out = re.findall(ptn, temp)
print(out)

['、', '、', '：', '，', '、', '、', '·', '，', '—', '—', '？', '—', '，', '，', '，', '·', '—', '—', '·', '—', '—', '、', '·', '—', '—', '·', '—', '—', '·', '、', '、', '，', '？', '、', '—', '、', '—', '—', '—', '—', '—', '—', '—', '—', '·', '—', '—', '·', '、', '·', '—', '—', '·', '！', '，', '，', '—', '—', '：', '：', '，', '？', '！', '？', '·', '·', '、', '，', '、', '、', '—', '·', '·', '—', '：', '：', '、', '·', '：', '·', '—', '—', '·', '—', '—', '—', '—', '：', '，', '、', '：', '，', '—', '—', '—', '，', '—', '—', '、', '、', '：', '、', '·', '·', '，', '·', '：', '—', '、', '、', '·', '·', '—', '：', '，', '·', '、', '、', '、', '：', '：', '，', '—', '—', '·', '—', '、', '—', '—', '—', '—', '、', '：', '、', '、', '：', '—', '·', '，', '、', '—', '，', '，', '—', '：', '—', '，', '—', '—', '—', '—', '—', '—', '：', '、', '、', '—', '—', '，', '·', '、', '：', '—', '—', '，', '！', '·', '、', '：', '—', '：', '：', '—', '—', '，', '·', '：', '·', '—', '—', '·', '—', '—', '、', '·', '·', '：', '，', '？', '：', '·', '·', '：', '：', '：', '·', '·', '：', '—', '—',

### 保存额外词典

In [29]:
with open('./custom_dict/vocab_book', 'w', encoding='utf-8') as w:
    for book in vocab:
        w.write(book+'\n')

## 保存结果

In [14]:
def split_data(data):
    length = len(data)
    len1 = int(length*6/10)
    len2 = int(length*8/10)
    len3 = int(length-1)
    return w.write(data[:len1]), w.write(data[len1:len2]), w.write(data[lenn2:len3])

split_data(raw)

with open('./data_proced/2_train', 'w', encoding="utf-8") as w:
    w.write(raw1)
with open('./data_proced/2_valid', 'w', encoding="utf-8") as w:
    w.write(raw2)
with open('./data_proced/2_test', 'w', encoding="utf-8") as w:
    w.write(raw3)

## 分词加空格

由于输入文本格式需要，要有分词和词典的建立

[自定义词典./custom_dict/vocab_book](#获取书名号中的词，作为分词单位)

### 方案一

In [15]:
import jieba
jieba.load_userdict('./custom_dict/vocab_book')
# 分词建立词典
cut1 = list(jieba.cut(raw1))
cut2 = list(jieba.cut(raw2))  # 进行分词
cut3 = list(jieba.cut(raw3))


vocab1 = set(cut1)
vocab2 = set(cut2)
vocab3 = set(cut3)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.530 seconds.
Prefix dict has been built succesfully.


In [16]:
vocab = list(vocab1|vocab2|vocab3)

In [17]:
print(len(vocab))

101127


In [18]:
# 删除多余的符号
print(punc)

{'。', '？', '：', '，', '！'}


In [19]:
for punc_ in punc:
    vocab.remove(punc_)

In [20]:
# 写入文件
with open('./final/vocab', 'w', encoding='utf-8') as w:
    w.write('\n'.join(vocab))
    w.write('\n')

#### 保存训练集、测试集、验证集

将分词后的文本，用空格为间隔进行保存。

In [21]:
# print(cut1)

In [22]:
# 写入训练集
with open('./final/train', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut1))
    w.write('\n')

In [23]:
# 写入验证集
with open('./final/valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut2))
    w.write('\n')

In [24]:
# 写入测试数据的正确标点数据
with open('./final/test_valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut3))
    w.write('\n')

#### 测试集变为无标点格式

In [25]:
print(punc)

{'。', '？', '：', '，', '！'}


In [26]:
tmp = ['，', 'as','a' , 'aa', '，']
tmp.remove('，')
print(tmp)

['as', 'a', 'aa', '，']


In [27]:
import copy
with open('./final/test', 'w', encoding='UTF-8') as w:
    cut3_copy = copy.deepcopy(cut3)
    cut3_copy = [x for x in cut3_copy if x not in punc]
    w.write(' '.join(cut3_copy))
    w.write('\n')

In [28]:
# 将测试验证数据集，变为无空格文本保存
with open('./final/test_valid', 'r', encoding='UTF-8') as f:
    with open('./final/test_valid_proc', 'w', encoding='UTF-8') as w:
        x = f.read()
        print(type(x))
        x = x.replace(' ', '')
        w.write(x)
# x

<class 'str'>


### 方案二

In [29]:
# 导入北大分词
import pkuseg

# 加载单词书
seg = pkuseg.pkuseg(user_dict='./custom_dict/vocab_book')           # 以默认配置加载模型

# 分词建立词典
cut2 = seg.cut(raw2)  # 进行分词
cut1 = seg.cut(raw1)
cut3 = seg.cut(raw3)
print(type(cut1))

<class 'list'>


In [30]:
# with open('./log/cut.log', 'w', encoding='utf-8') as w:
#     print(cut1+cut2+cut3, file=w)

vocab1 = set(cut1)
vocab2 = set(cut2)
vocab3 = set(cut3)

vocab = list(vocab1 | vocab2 | vocab3)

In [31]:
print(len(vocab))

110655


In [32]:
# 删除多余的符号
print(punc)

{'。', '？', '：', '，', '！'}


In [33]:
for punc_ in punc:
    vocab.remove(punc_)

In [34]:
# 写入文件
with open('./final2/vocab', 'w', encoding='utf-8') as w:
    w.write('\n'.join(vocab))
    w.write('\n')

#### 保存训练集、测试集、验证集

将分词后的文本，用空格为间隔进行保存。

In [35]:
# print(cut1)

In [36]:
# 写入训练集
with open('./final2/train', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut1))
    w.write('\n')

In [37]:
# 写入验证集
with open('./final2/valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut2))
    w.write('\n')

In [38]:
# 写入测试数据的正确标点数据
with open('./final2/test_valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut3))
    w.write('\n')

#### 测试集变为无标点格式

In [39]:
print(punc)

{'。', '？', '：', '，', '！'}


In [40]:
tmp = ['，', 'as','a' , 'aa', '，']
tmp.remove('，')
print(tmp)

['as', 'a', 'aa', '，']


In [41]:
import copy
with open('./final2/test', 'w', encoding='UTF-8') as w:
    cut3_copy = copy.deepcopy(cut3)
    cut3_copy = [x for x in cut3_copy if x not in punc]
    w.write(' '.join(cut3_copy))
    w.write('\n')

In [42]:
# 将测试验证数据集，变为无空格文本保存
with open('./final2/test_valid', 'r', encoding='UTF-8') as f:
    with open('./final2/test_valid_proc', 'w', encoding='UTF-8') as w:
        x = f.read()
        print(type(x))
        x = x.replace(' ', '')
        w.write(x)
# x

<class 'str'>


### 方案三

In [43]:
# from jiayan import load_lm
# from jiayan import CharHMMTokenizer

# text = '是故内圣外王之道，暗而不明，郁而不发，天下之人各为其所欲焉以自为方。'

# lm = load_lm('jiayan.klm')
# tokenizer = CharHMMTokenizer(lm)
# print(list(tokenizer.tokenize(text)))

In [44]:
from jiayan import load_lm
from jiayan import CharHMMTokenizer
lm = load_lm('jiayan.klm')
tokenizer = CharHMMTokenizer(lm)

In [45]:
# 分词建立词典
cut1 = list(tokenizer.tokenize(raw1))
cut2 = list(tokenizer.tokenize(raw2)) 
cut3 = list(tokenizer.tokenize(raw3))
print(type(cut1))

<class 'list'>


In [46]:
# with open('./log/cut.log', 'w', encoding='utf-8') as w:
#     print(cut1+cut2+cut3, file=w)

vocab1 = set(cut1)
vocab2 = set(cut2)
vocab3 = set(cut3)

vocab = list(vocab1 | vocab2 | vocab3)

In [47]:
print(len(vocab))

42938


In [48]:
# 删除多余的符号
print(punc)

{'。', '？', '：', '，', '！'}


In [49]:
for punc_ in punc:
    vocab.remove(punc_)

In [50]:
# 写入文件
with open('./final3/vocab', 'w', encoding='utf-8') as w:
    w.write('\n'.join(vocab))
    w.write('\n')

#### 保存训练集、测试集、验证集

将分词后的文本，用空格为间隔进行保存。

In [51]:
# print(cut1)

In [52]:
# 写入训练集
with open('./final3/train', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut1))
    w.write('\n')

In [53]:
# 写入验证集
with open('./final3/valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut2))
    w.write('\n')

In [54]:
# 写入测试数据的正确标点数据
with open('./final3/test_valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut3))
    w.write('\n')

#### 测试集变为无标点格式

In [55]:
print(punc)

{'。', '？', '：', '，', '！'}


In [56]:
tmp = ['，', 'as','a' , 'aa', '，']
tmp.remove('，')
print(tmp)

['as', 'a', 'aa', '，']


In [57]:
import copy
with open('./final3/test', 'w', encoding='UTF-8') as w:
    cut3_copy = copy.deepcopy(cut3)
    cut3_copy = [x for x in cut3_copy if x not in punc]
    w.write(' '.join(cut3_copy))
    w.write('\n')

In [58]:
# 将测试验证数据集，变为无空格文本保存
with open('./final3/test_valid', 'r', encoding='UTF-8') as f:
    with open('./final3/test_valid_proc', 'w', encoding='UTF-8') as w:
        x = f.read()
        print(type(x))
        x = x.replace(' ', '')
        w.write(x)
# x

<class 'str'>


### 方案四

In [59]:
for i in range(4):
    print(i)

0
1
2
3


In [60]:
# 分词建立词典
cut1 = [raw1[i] for i in range(len(raw1))]
cut2 = [raw2[i] for i in range(len(raw2))]
cut3 = [raw3[i] for i in range(len(raw3))]
print(type(cut1))

<class 'list'>


In [61]:
# with open('./log/cut.log', 'w', encoding='utf-8') as w:
#     print(cut1+cut2+cut3, file=w)

vocab1 = set(cut1)
vocab2 = set(cut2)
vocab3 = set(cut3)

vocab = list(vocab1 | vocab2 | vocab3)

In [62]:
print(len(vocab))

6637


In [63]:
# 删除多余的符号
print(punc)

{'。', '？', '：', '，', '！'}


In [64]:
for punc_ in punc:
    vocab.remove(punc_)

In [65]:
# 写入文件
with open('./final4/vocab', 'w', encoding='utf-8') as w:
    w.write('\n'.join(vocab))
    w.write('\n')

#### 保存训练集、测试集、验证集

将分词后的文本，用空格为间隔进行保存。

In [66]:
# print(cut1)

In [67]:
# 写入训练集
with open('./final4/train', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut1))
    w.write('\n')

In [68]:
# 写入验证集
with open('./final4/valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut2))
    w.write('\n')

In [69]:
# 写入测试数据的正确标点数据
with open('./final4/test_valid', 'w', encoding='UTF-8') as w:
    w.write(' '.join(cut3))
    w.write('\n')

#### 测试集变为无标点格式

In [70]:
print(punc)

{'。', '？', '：', '，', '！'}


In [71]:
tmp = ['，', 'as','a' , 'aa', '，']
tmp.remove('，')
print(tmp)

['as', 'a', 'aa', '，']


In [72]:
import copy
with open('./final4/test', 'w', encoding='UTF-8') as w:
    cut3_copy = copy.deepcopy(cut3)
    cut3_copy = [x for x in cut3_copy if x not in punc]
    w.write(' '.join(cut3_copy))
    w.write('\n')

In [73]:
# 将测试验证数据集，变为无空格文本保存
with open('./final4/test_valid', 'r', encoding='UTF-8') as f:
    with open('./final4/test_valid_proc', 'w', encoding='UTF-8') as w:
        x = f.read()
        print(type(x))
        x = x.replace(' ', '')
        w.write(x)
# x

<class 'str'>
